# Tensorflow Estimator [reference](https://ai.google/research/pubs/pub46369)

To work with the estimators, we’ve to become comfortable with two new concepts
1. Feature columns:
    Feature columns are specifications for how the model should interpret the input data
    
2. Input Function:
    Input functions are used for passing input data to the model for training and evaluation
    
Basically workflow how to use [tensorflow estimator](https://medium.com/learning-machine-learning/introduction-to-tensorflow-estimators-part-1-39f9eb666bc7) including some step:
1. Loading the libraries and dataset.
2. Data proprocessing.
3. Defining the feature columns.
4. Building input function.
5. Model instantiation, training and evaluation.
6. Generating prediction.
7. Visualizing the model and the loss metrics using Tensorboard.

![feature_columns](./Basic_TF_Estimator_presentation/feature_columns.png)

<b>The code below will re-implemented the [tf.estimator tutorial](https://guillaumegenthial.github.io/introduction-tensorflow-estimator.html) from [Guillaume Genthial blog](https://guillaumegenthial.github.io/)</b>

# Global configuration


# Model
We do regression function f(x1, x2) = 2x
The model will learn to define two parameter a, b in linear model a * x1 + b * x2

so fisrt we construct model_fn

In [1]:
import tensorflow as tf
import os
print(tf.__version__)

1.13.1


In [2]:
def model_fn(features, labels, model, params):
    if isinstance(features, dict):
        features = features["feature"]
        
    predictions = tf.layers.dense(features, 1)
    
    if mode == tf.layers.Modelkeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    else:
        loss = tf.nn.l2_loss(predictions - labels)
        if mode == tf.layers.Modelkeys.EVAL:
            return tf.estimator.EstimatorSpec(mode, loss=loss)
        
        elif mode == tf.layers.Modelkeys.TRAIN:
            train_op = tf.train.Adamoptimizer(learning_rate=0.5).minimize(loss, global_stept=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
        
        else:
            raise NotImplementedError()
        
            

# Train the Estimator
To train our model, we generate fake data


In [3]:
def train_generator_fn():
    for number in range(100):
        yield [number, number] [2 * number]
        
def train_input_fn():
    shapes, types = (2, 1), (tf.float32, tf.float32)
    dataset = tf.data.Dataset.from_generator(train_generator_fn, output_types=types, output_shapes = shapes)
    dataset = dataset.batch(20).repeat(200)
    return dataset


In [4]:
estimator = tf.estimator.Estimator(model_fn, "model", params={})
estimator.train(train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feb987f9898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


ValueError: model_fn (<function model_fn at 0x7fec313b5378>) has following not expected args: ['model']